In [11]:
%%time
import pandas as pd
import scipy.sparse
from scipy.sparse import csr_matrix
import numpy as np
from scipy.sparse.linalg import inv
df_input = pd.read_csv("web-Google.txt", comment = "#", sep = '\t',dtype = int, names = ['from','to'])
display(df_input)

,from,to
0,0,11342
1,0,824020
2,0,867923
3,0,891835
4,11342,0
...,...,...
5105034,916425,547323
5105035,916425,604833
5105036,916425,632916
5105037,916425,637936


Wall time: 2.31 s


In [12]:
df_count_rank = df_input.drop_duplicates().groupby('from').count().rename(columns={"to":"count"})
df_count_rank['rank'] = 1
df_count_rank['contribution'] = df_count_rank['rank']/df_count_rank['count']
df_count_rank = pd.merge(df_input, df_count_rank, on = 'from')
display(df_count_rank)

,from,to,count,rank,contribution
0,0,11342,4,1,0.250000
1,0,824020,4,1,0.250000
2,0,867923,4,1,0.250000
3,0,891835,4,1,0.250000
4,11342,0,14,1,0.071429
...,...,...,...,...,...
5105034,916425,547323,12,1,0.083333
5105035,916425,604833,12,1,0.083333
5105036,916425,632916,12,1,0.083333
5105037,916425,637936,12,1,0.083333


In [13]:
#build sparse matrix
row = df_count_rank["to"].to_numpy()
col = df_count_rank["from"].to_numpy()
data = df_count_rank["contribution"].to_numpy()
s_mat = csr_matrix((data, (col, row)))
print(s_mat)
#display(pd.DataFrame.sparse.from_spmatrix(mat))

  (0, 11342)	0.25
  (0, 824020)	0.25
  (0, 867923)	0.25
  (0, 891835)	0.25
  (1, 53051)	0.1
  (1, 203402)	0.1
  (1, 223236)	0.1
  (1, 276233)	0.1
  (1, 552600)	0.1
  (1, 569212)	0.1
  (1, 635575)	0.1
  (1, 748615)	0.1
  (1, 862566)	0.1
  (1, 893884)	0.1
  (2, 30957)	0.09090909090909091
  (2, 357310)	0.09090909090909091
  (2, 423174)	0.09090909090909091
  (2, 430119)	0.09090909090909091
  (2, 462435)	0.09090909090909091
  (2, 472889)	0.09090909090909091
  (2, 565424)	0.09090909090909091
  (2, 581609)	0.09090909090909091
  (2, 597621)	0.09090909090909091
  (2, 644135)	0.09090909090909091
  (2, 858904)	0.09090909090909091
  :	:
  (916424, 417728)	0.25
  (916424, 565078)	0.25
  (916424, 661130)	0.25
  (916425, 116832)	0.08333333333333333
  (916425, 197330)	0.08333333333333333
  (916425, 218829)	0.08333333333333333
  (916425, 229195)	0.08333333333333333
  (916425, 367498)	0.08333333333333333
  (916425, 379452)	0.08333333333333333
  (916425, 521152)	0.08333333333333333
  (916425, 547323)	0.0

In [14]:
#x= scipy.sparse.eye(916428) - 0.85 * mat
#y = csr_matrix(np.full((5,1),0.15))
#xIn = scipy.sparse.linalg.inv(x)
#p1 = scipy.sparse.linalg.inv(mat)
#print(p1)
#p2 = y.A
#print(p1)
#print(p2)
#print(np.matmul(p1,p2))

In [15]:
#build the damp vector
initial_vector = csr_matrix(np.ones(s_mat.shape[0])/s_mat.shape[0])
#Calculate PageRank
beta = 0.85
# The result of product of matrix and vector after each iteration
pg = initial_vector.T

for i in range(50):
    pg = beta*s_mat.T*pg +(1-beta)*initial_vector.T
pg = pd.DataFrame(pg.toarray())

# Sort the result
pg = pg.sort_values(by=0, ascending = False)
pg.to_csv('out.csv')

In [16]:
pg

,0
597621,6.157352e-04
41909,6.140082e-04
163075,6.025874e-04
537039,5.991416e-04
384666,5.245271e-04
...,...
184350,1.636790e-07
403434,1.636790e-07
403433,1.636790e-07
403432,1.636790e-07
